In [1]:
import json
import time
import datetime
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm

with open('tmdb_api.json', 'r') as rd_f:
    tokens = json.load(rd_f)

In [2]:
#headers for url request
headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {tokens['token']}"}

List of All Genres

In [3]:
#get genre ids
genres_url = "https://api.themoviedb.org/3/genre/movie/list?language=en"

#send_request to api
response = requests.get(genres_url, headers=headers)

#extract genres to api
genres_list = {genre['id']: genre['name'] for genre in response.json()['genres']}
print(genres_list)

{28: 'Action', 12: 'Adventure', 16: 'Animation', 35: 'Comedy', 80: 'Crime', 99: 'Documentary', 18: 'Drama', 10751: 'Family', 14: 'Fantasy', 36: 'History', 27: 'Horror', 10402: 'Music', 9648: 'Mystery', 10749: 'Romance', 878: 'Science Fiction', 10770: 'TV Movie', 53: 'Thriller', 10752: 'War', 37: 'Western'}


In [4]:
#url for tv shows
# tv_url = "https://api.themoviedb.org/3/discover/tv?first_air_date.gte=1990-01-01&include_adult=false&include_null_first_air_dates=false&language=en-US&page=1&sort_by=popularity.desc&with_original_language=en"

#url for movies
# movie_url = f"https://api.themoviedb.org/3/discover/movie?include_adult=false&include_video=false&language=en-US&page={page}&release_date.gte={gte}&release_date.lte={lte}&with_origin_country=US&sort_by=release_date.desc"

Page Number Cannot Be Higher than 500

In [5]:
#number of years to look back
num_years = 20
#x year increments
increment = 0.5

#year of 20 years ago
twenty_years = datetime.date.today().year - num_years

#increments for the last 20 years
twenty_years_list = [str(datetime.date(int(twenty_years + val), int(1 + 12 * (val % 1)), 1))
                     for val in np.arange(0, num_years, increment).tolist()]
print(twenty_years_list)

['2003-01-01', '2003-07-01', '2004-01-01', '2004-07-01', '2005-01-01', '2005-07-01', '2006-01-01', '2006-07-01', '2007-01-01', '2007-07-01', '2008-01-01', '2008-07-01', '2009-01-01', '2009-07-01', '2010-01-01', '2010-07-01', '2011-01-01', '2011-07-01', '2012-01-01', '2012-07-01', '2013-01-01', '2013-07-01', '2014-01-01', '2014-07-01', '2015-01-01', '2015-07-01', '2016-01-01', '2016-07-01', '2017-01-01', '2017-07-01', '2018-01-01', '2018-07-01', '2019-01-01', '2019-07-01', '2020-01-01', '2020-07-01', '2021-01-01', '2021-07-01', '2022-01-01', '2022-07-01']


In [6]:
total_movies = []
failed = []

#iterate through all time ranges and pages
for i in range(len(twenty_years_list)):
    #set api filters
    if i == len(twenty_years_list) - 1:
        page, gte, lte = 1, twenty_years_list[i], str(datetime.date.today())
    else:
        page, gte, lte = 1, twenty_years_list[i], twenty_years_list[i+1]
    #add one day to greater than date
    gte = f"{gte[:-1]}{int(gte[-1]) + 1}"
    
    #format url for api call
    url = f"https://api.themoviedb.org/3/discover/movie?include_adult=false&include_video=false&language=en-US&page={page}&release_date.gte={gte}&release_date.lte={lte}&with_origin_country=US&sort_by=release_date.desc"
    
    #send_request to api
    response = requests.get(url, headers=headers)

    #extract total pages
    total_pages = response.json()['total_pages']
    print(f"Range: ({gte}, {lte}) | Total pages: {total_pages:,}")
    
    for x in tqdm(range(1, total_pages if total_pages < 501 else 500)):
        try:
            #format url for api call
            url = f"https://api.themoviedb.org/3/discover/movie?include_adult=false&include_video=false&language=en-US&page={x}&release_date.gte={gte}&release_date.lte={lte}&with_origin_country=US&sort_by=release_date.asc"
            #send request to api
            response = requests.get(url, headers=headers)
            #append results to overall list
            total_movies += response.json()['results']
        except:
            failed.append(f"Range: ({gte}, {lte}) | Page: {x:,}")

Range: (2003-01-02, 2003-07-01) | Total pages: 122


100%|████████████████████████████████████████████████████████████████████████████████| 121/121 [00:35<00:00,  3.44it/s]


Range: (2003-07-02, 2004-01-01) | Total pages: 198


100%|████████████████████████████████████████████████████████████████████████████████| 197/197 [01:18<00:00,  2.51it/s]


Range: (2004-01-02, 2004-07-01) | Total pages: 131


...

100%|████████████████████████████████████████████████████████████████████████████████| 499/499 [02:55<00:00,  2.84it/s]


Range: (2022-01-02, 2022-07-01) | Total pages: 385


100%|████████████████████████████████████████████████████████████████████████████████| 384/384 [01:28<00:00,  4.32it/s]


Range: (2022-07-02, 2023-12-06) | Total pages: 1,014


100%|████████████████████████████████████████████████████████████████████████████████| 499/499 [01:44<00:00,  4.79it/s]


In [7]:
#turn into dataframe
total_movies = pd.DataFrame(total_movies)

#remove any duplicates
total_movies = total_movies.drop_duplicates('title', keep='first').reset_index(drop=True)

#return to original format
total_movies = total_movies.to_dict(orient='records')

#remove movies with no poster path
total_movies = [movie for movie in total_movies if movie['poster_path'] is not None]

In [8]:
#map genre ids to genre text and complete poster path
for i in tqdm(range(len(total_movies))):
    total_movies[i]['genres'] = [genres_list[id_] for id_ in total_movies[i]['genre_ids']]
    total_movies[i]['poster_path'] = f"https://image.tmdb.org/t/p/w500/{total_movies[i]['poster_path']}"
    
print(f"Total Movies: {len(total_movies):,}")

100%|██████████████████████████████████████████████████████████████████████| 134938/134938 [00:00<00:00, 197172.28it/s]

Total Movies: 134,938


In [9]:
with open('../movies_database.json', 'w') as wt_f:
    json.dump(total_movies, wt_f)